In [1]:
# import dependencies
import spotipy
import spotipy.util as util
import pandas as pd
import pprint

## Grab csv files dataframe

In [484]:
top_2017 = pd.read_csv("assets/data/top2017.csv", encoding='latin-1')
top_2018 = pd.read_csv("assets/data/top2018.csv", encoding='unicode_escape')
top_2019 = pd.read_csv("assets/data/top2019.csv", encoding='unicode_escape')

### create search list of each year

In [3]:
# initialize list to scrape
search_data = []

# pull tracks and artists
tracks = list(top_2019['Track.Name'])
artists = list(top_2019['Artist.Name'])
# add each list to search_data
for i in range(len(tracks)):
    search_data.append([tracks[i], artists[i]])



In [485]:
def search_by_track(track, artist):
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name 
    result = sp.search(q=track, type='track')
    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album 
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = [artist, track, result['tracks']['items'][0]['id']]
            return value
    return None

def search_by_artist(artist, track):
    ''' returns correct track by artist to search'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
  
    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']
    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        # matches enough, match found
        if track[:2] == possible_song[:2] or track[-3:] == possible_song[-3:]:
            track = possible_song
            return track
    return None


def grab_genre(artist):
    ''' returns a genre of artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    genre = sp.artist(artist_id)['genres']
    return genre
 

In [5]:
#https://open.spotify.com/user/1298997744?si=ijwp9NZOQoyjO9qOZliclw
SPOTIPY_CLIENT_ID='7655188a18144343b07e44965cc38310'
SPOTIPY_CLIENT_SECRET='2bec9b47853e4ea3a419418a211ec45d'
SPOTIPY_REDIRECT_URI='https://google.com/'
scope = 'user-library-read'
# Grab username
username = "1298997744"

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

results = []

# Create our spotify object
for query in search_data:
    if token:
        # create track & artist search variables
        track = query[0]
        artist = query[1]
        genre = grab_genre(artist)

        # run search by track to find results
        value = search_by_track(track, artist)

        # if search_by_track returns none, use search_by_artist.
        # else append to results
        if value == None:
            correct_track = search_by_artist(artist, track)
            # run search_by_track again
            value = search_by_track(correct_track, artist)
            value.append(genre)
            results.append(value)
        else:
            value.append(genre)
            results.append(value)   
    else:
        print("Can't get token for", username)

In [6]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)

top_2019_data = []
for each in results:
    features = sp.audio_features(each[2])
    data = features[0]
    data['name'] = each[1]
    data['artists'] = each[0]
    data['year'] = '2019'
    data['genre'] = each[3]
    top_2019_data.append(data)

In [7]:
top2019_df = pd.DataFrame.from_dict(top_2019_data)
top_2019_df = top2019_df.drop(columns=['type', 'uri','track_href','analysis_url'])
top_2019_df = top_2019_df[['id','name','artists','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','year','genre']]
top_2019_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019,"[canadian pop, pop, post-teen pop, viral pop]"
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019,"[latin, reggaeton flow, trap latino]"
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019,"[dance pop, pop, post-teen pop]"
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019,"[pop, uk pop]"
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019,"[dfw rap, melodic rap, rap]"
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019,"[pop, uk pop]"
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019,[melodic rap]
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019,"[pop, uk pop]"
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019,"[country rap, lgbtq+ hip hop, pop rap]"
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019,"[electropop, pop]"


In [ ]:
#top_2019_df.to_csv('assets/data/top2019fixed.csv', index=False)
top_2019_df.to_excel('assets/data/top2019fixed2.xlsx', index=False)

In [ ]:
final_df = pd.read_excel("assets/data/top2019fixed2.xlsx")
final_df

### finds genre of artist

In [486]:
def add_genre(dataframe):
    '''takes in a dataframe, adds a genre column by artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # grab artist column for Spotipy search
    artists = list(dataframe['artists'])  
   
    # initialize genre column
    genres = []

    for person in artists:
        # create spotipy object
        sp = spotipy.Spotify(auth=token)
        # search by artist name
        result = sp.search(q=person, type='artist')
        try:
            # grab artist id
            artist_id = result['artists']['items'][0]['id']
            # search artist top tracks
            genre = sp.artist(artist_id)['genres']
            genres.append(genre)
        except IndexError:
            i = dataframe.index[dataframe['artists'] == person]
            song_df = dataframe.loc[i, 'name']
            song_list = song_df.values
            song = song_list[0]
            result = sp.search(q=song, type='track')
            artist = result['tracks']['items'][0]['artists'][0]['name']
            result = sp.search(q=artist, type='artist')
            genre = result['artists']['items'][0]['genres']
            genres.append(genre)
                   
    dataframe['genre'] = genres


In [505]:
add_genre(top_2017)
#add_genre(top_2018)

In [11]:
print(top_2017_df)

None


In [60]:
dfs = [top_2019_df, top_2017, top_2018]

In [61]:
all_genres = []

for df in dfs:
    genres_in_year = list(df['genre'])
    for artist_genre in genres_in_year:
        for genre in artist_genre:
            if genre not in all_genres:
                all_genres.append(genre)
            

In [63]:
all_genres

94

In [65]:
all_genres_dict = {}
for genre in all_genres:
    all_genres_dict[genre] = [] 
    
all_genres_dict

{'canadian pop': [],
 'pop': [],
 'post-teen pop': [],
 'viral pop': [],
 'latin': [],
 'reggaeton flow': [],
 'trap latino': [],
 'dance pop': [],
 'uk pop': [],
 'dfw rap': [],
 'melodic rap': [],
 'rap': [],
 'country rap': [],
 'lgbtq+ hip hop': [],
 'pop rap': [],
 'electropop': [],
 'reggaeton': [],
 'electronic trap': [],
 'panamanian pop': [],
 'canadian hip hop': [],
 'hip hop': [],
 'toronto rap': [],
 'r&b': [],
 'escape room': [],
 'minnesota hip hop': [],
 'trap queen': [],
 'pop house': [],
 'australian pop': [],
 'latin hip hop': [],
 'tropical': [],
 'tropical house': [],
 'atl hip hop': [],
 'atl trap': [],
 'gangster rap': [],
 'trap': [],
 'big room': [],
 'edm': [],
 'progressive house': [],
 'boy band': [],
 'pop rock': [],
 'alternative r&b': [],
 'r&b en espanol': [],
 'brostep': [],
 'progressive electro house': [],
 'latin pop': [],
 'puerto rican pop': [],
 'miami hip hop': [],
 'southern hip hop': [],
 'conscious hip hop': [],
 'west coast rap': [],
 'philly 

In [67]:
#genres_to_songs = top_2019_df[['id','name','genre']]


for df in dfs:
    genres_to_songs = df[['id','name','genre']]
    for index, data in genres_to_songs.iterrows():
        # data['id'] & data['name']
        for each in data['genre']:
            all_genres_dict[each].append([data['id'],data['name']])

In [53]:
for index, data in genres_to_songs.iterrows():
    # data['id'] & data['name']
    for each in data['genre']:
        all_genres_dict[each].append([data['id'],data['name']])
        

In [68]:
all_genres_dict

{'canadian pop': [['0TK2YIli7K1leLovkQiNik', 'Señorita'],
  ['5ry2OE6R2zPQFDO85XkgRb', 'Money In The Grave (Drake ft. Rick Ross)'],
  ['6LsAAHotRLMOHfCsSfYCsz', "If I Can't Have You"],
  ['79cuOz3SPQTuFrp8WgftA', "There's Nothing Holdin' Me Back"],
  ['5GXAXm5YOmYT0kL5jHvYB', 'I Feel It Coming'],
  ['5aAx2yezTd8zXrkmtKl66', 'Starboy'],
  ['7hDc8b7IXETo14hHIHdnh', 'Passionfruit'],
  ['343YBumqHu19cGoGARUTs', 'Fake Love'],
  ['1xznGGDReH1oQq0xzbwXa', 'One Dance'],
  ['7nZmah2llfvLDiUjm0kiy', 'Friends (with BloodPopÂ®)'],
  ['6DCZcSspjsKoFjzjrWoCd', "God's Plan"],
  ['2G7V7zsVDxg1yRsu7Ew9R', 'In My Feelings'],
  ['3CA9pLiwRIGtUBiMjbZmR', 'Nice For What'],
  ['1gm616Plq4ScqNi7TVkZ5', 'Call Out My Name'],
  ['0TlLq3lA83rQOYtrqBqSc', 'Nonstop'],
  ['2QZ7WLBE8h2y1Y5Fb8RYb', 'In My Blood'],
  ['77UjLW8j5UAGAGVGhR5oU', 'Pray For Me (with Kendrick Lamar)']],
 'pop': [['0TK2YIli7K1leLovkQiNik', 'Señorita'],
  ['0Ryd8975WihbObpp5cPW1t', 'boyfriend (with Social House)'],
  ['4evmHXcjt3bTUHD1cvny97'

In [185]:
filtered_genres = {}

for k,v in all_genres_dict.items():
    if len(v) >= 4:
        filtered_genres[k] = v

In [197]:
        
for k in sorted(filtered_genres, key=lambda k: len(filtered_genres[k]), reverse=True):
    print(k, len(filtered_genres[k]))

pop 129
dance pop 69
post-teen pop 54
rap 52
pop rap 43
tropical house 38
latin 34
edm 31
reggaeton 28
uk pop 25
hip hop 25
trap 19
canadian pop 17
tropical 17
melodic rap 16
electro house 13
electropop 12
latin pop 12
uk dance 11
dfw rap 10
atl hip hop 10
canadian hip hop 9
miami hip hop 9
southern hip hop 9
toronto rap 8
latin hip hop 8
big room 8
progressive house 8
viral pop 7
reggaeton flow 7
brostep 7
house 7
emo rap 7
trap latino 6
pop rock 6
progressive electro house 6
modern rock 6
alternative r&b 5
puerto rican pop 5
conscious hip hop 5
uk funky 5
canadian contemporary r&b 5
boy band 4
west coast rap 4
mexican pop 4


In [181]:
len(filtered_genres)
list(filtered_genres.keys())

['canadian pop',
 'pop',
 'latin',
 'dance pop',
 'dfw rap',
 'melodic rap',
 'rap',
 'pop rap',
 'electropop',
 'canadian hip hop',
 'hip hop',
 'toronto rap',
 'tropical',
 'tropical house',
 'atl hip hop',
 'trap',
 'big room',
 'edm',
 'progressive house',
 'boy band',
 'pop rock',
 'alternative r&b',
 'brostep',
 'progressive electro house',
 'puerto rican pop',
 'miami hip hop',
 'southern hip hop',
 'conscious hip hop',
 'west coast rap',
 'electro house',
 'uk dance',
 'uk funky',
 'modern rock',
 'canadian contemporary r&b',
 'emo rap']

In [229]:
genre_checker = filtered_genres['tropical house']
genre_checker

[['3g0mEQx3NTanacLseoP0Gw', 'Takeaway'],
 ['7Feaw9WAEREY0DUOSXJLOM',
  'Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy)'],
 ['6oJ6le65B3SEqPwMRNXWjY', 'Higher Love'],
 ['2oejEp50ZzPuQTQ6v54Evp', 'Call You Mine'],
 ['6RUKPb4LETWmmr3iAEQkt', 'Something Just Like This'],
 ['3eR23VReFzcdmS7TYCrhC', "It Ain't Me (with Selena Gomez)"],
 ['72jbDTw1piOOj770jWNea', 'Paris'],
 ['0dA2Mk56wEzDgegdC6R17', 'Stay (with Alessia Cara)'],
 ['5knuzwU65gJK7IF5yJsua', 'Rockabye (feat. Sean Paul & Anne-Marie)'],
 ['3ebXMykcMXOcLeJ9xZ17X', 'Scared to Be Lonely'],
 ['7BKLCZ1jbUBVqRi2FVlTV', 'Closer'],
 ['1x5sYLZiu9r5E43kMlt9f', 'Symphony (feat. Zara Larsson)'],
 ['7tr2za8SQg2CI8EDgrdtN', 'Slide'],
 ['6EpRaXYhGOB3fj4V2uDkM', 'Strip That Down'],
 ['5bcTCxgc7xVfSaMV3RuVk', 'Feels'],
 ['0NiXXAI876aGImAd6rTj8', 'Mama'],
 ['6HUnnBwYZqcED1eQztxMB', 'Solo Dance'],
 ['3PEgB3fkiojxms35ntsTg', 'More Than You Know'],
 ['00lNx0OcTJrS3MKHcB80H', "You Don't Know Me - Radio Edit"],
 ['1louJpMmzEicAn7lzDalP', '

In [203]:
for song in genre_checker:
    try:
        print(filtered_genres['pop rap'].index(song))
    except ValueError:
        print('Couldnt find song:', song)
        

Couldnt find song: ['0t3ZvGKlmYmVsDzBJAXK8C', 'Goodbyes (Feat. Young Thug)']
1
2
Couldnt find song: ['3KkXRkHbMCARz0aVfEt68P', 'Sunflower - Spider-Man: Into the Spider-Verse']
6
7
Couldnt find song: ['7KXjTSCq5nL1LoYtL7XAw', 'HUMBLE.']
8
9
10
Couldnt find song: ['3a1lNhkSLSkpJE4MSHpDu', 'Congratulations']
12
13
14
Couldnt find song: ['7wGoVu4Dady5GV0Sv4UIs', 'rockstar']
15
16
17
18
19
Couldnt find song: ['6HZILIRieu8S0iqY8kIKh', 'DNA.']
20
Couldnt find song: ['6gBFPUFcJLzWGx4lenP6h', 'goosebumps']
23
24
25
Couldnt find song: ['0e7ipj03S05BNilyu5bRz', 'rockstar (feat. 21 Savage)']
Couldnt find song: ['3swc6WTsr7rl9DqQKQA55', 'Psycho (feat. Ty Dolla $ign)']
26
Couldnt find song: ['7dt6x5M1jzdTEt8oCbisT', 'Better Now']
27
28
Couldnt find song: ['3GCdLUSnKSMJhs4Tj6CV3', 'All The Stars (with SZA)']
29
30
Couldnt find song: ['75ZvA4QfFiZvzhj2xkaWA', 'I Fall Apart']
Couldnt find song: ['1cS0TgbR263ey9jn0MwD2', 'River (feat. Ed Sheeran)']
Couldnt find song: ['2xLMifQCjDGFmkHkpNLD9', 'SICKO MOD

In [182]:
del filtered_genres['house']

KeyError: 'house'

KeyError: 'post-teen pop'

In [103]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)
# search by artist name
result = sp.audio_analysis('0TK2YIli7K1leLovkQiNik')
result

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1576651784,
  'analysis_time': 5.72235,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 4210668,
  'duration': 190.96,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.18331,
  'start_of_fade_out': 188.34866,
  'loudness': -6.039,
  'tempo': 116.947,
  'tempo_confidence': 0.673,
  'time_signature': 4,
  'time_signature_confidence': 1.0,
  'key': 9,
  'key_confidence': 0.445,
  'mode': 0,
  'mode_confidence': 0.526,
  'codestring': 'eJxVmgmy3TgOBK-iI3Bf7n-xySw-tz0d0Y5PPokEsRQKoNasvZ5Z71e-NespdY_7rVG-Wsvuu5TJX2d8bd5a1qj9q4Nfe9_njumv666vMpyzfvXOxqPn3rvq-dpYebPN2kr92m7sUs-tvdT5tbuXm-5daulfb719l1VHX6t9fZzz7VVqn23vr-_BUnXXfvr9RinlY8Y3yzfaZKE6Zj9tMdzrfKMj8Eak2Q6PIkOt845vzr6-03vneItf922cCNl3461VxuHXtitzaoTlWrlj38Nx1qkctbR9yi38vMtuPNNQweA8u7fzDj

In [488]:
top_2018

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.449,7,-9.211,1,0.1090,0.033200,0.000083,0.5520,0.3570,77.169,198973,4,2018,"[canadian hip hop, canadian pop, hip hop, pop ..."
1,3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.613,8,-4.880,1,0.1450,0.258000,0.003720,0.1230,0.4730,75.023,166606,4,2018,"[emo rap, miami hip hop]"
2,0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5,-6.090,0,0.0898,0.117000,0.000066,0.1310,0.1400,159.847,218147,4,2018,"[dfw rap, melodic rap, rap]"
3,3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8,-8.011,1,0.1170,0.580000,0.000000,0.1120,0.4390,140.124,221440,4,2018,"[dfw rap, melodic rap, rap]"
4,2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.626,1,-5.833,1,0.1250,0.058900,0.000060,0.3960,0.3500,91.030,217925,4,2018,"[canadian hip hop, canadian pop, hip hop, pop ..."
5,7dt6x5M1jzdTEt8oCbisT,Better Now,Post Malone,0.680,0.563,10,-5.843,1,0.0454,0.354000,0.000000,0.1360,0.3740,145.028,231267,4,2018,"[dfw rap, melodic rap, rap]"
6,58q2HKrzhC3ozto2nDdN4,I Like It,Cardi B,0.816,0.726,5,-3.998,0,0.1290,0.099000,0.000000,0.3720,0.6500,136.048,253390,4,2018,"[pop, pop rap, rap]"
7,7ef4DlsgrMEH11cDZd32M,One Kiss (with Dua Lipa),Calvin Harris,0.791,0.862,9,-3.240,0,0.1100,0.037000,0.000022,0.0814,0.5920,123.994,214847,4,2018,"[dance pop, edm, electro house, house, pop, pr..."
8,76cy1WJvNGJTj78UqeA5z,IDGAF,Dua Lipa,0.836,0.544,7,-5.975,1,0.0943,0.040300,0.000000,0.0824,0.5100,97.028,217947,4,2018,"[dance pop, pop, post-teen pop, uk pop]"
9,08bNPGLD8AhKpnnERrAc6,FRIENDS,Marshmello,0.626,0.880,9,-2.384,0,0.0504,0.205000,0.000000,0.1280,0.5340,95.079,202621,4,2018,"[brostep, progressive electro house]"


In [489]:
final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']

In [490]:
def remove_unpopular_genres(dataframe):
    final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']
    
    for i, genres in dataframe['genre'].iteritems():
        index = 0
        while index < len(genres):
            if genres[index] in final_genres:
                index += 1
            else:
                del(genres[index])
            


In [491]:
def add_genre_other(dataframe):
    for i, genres in dataframe['genre'].iteritems():
        if len(genres) == 0:
            genres.append('other')

In [492]:
def del_mult_genres(dataframe):
    for i, song_genres in dataframe['genre'].iteritems():
        if len(song_genres) > 1:
            index = 0
            while index < len(final_genres):
                if final_genres[index] in song_genres:
                    dataframe.loc[i,'genre'] = final_genres[index]
                    break
                else:
                    index += 1
        else:
            dataframe.loc[i,'genre'] = song_genres

In [506]:
#del_mult_genres(top_2017)
remove_unpopular_genres(top_2017)

In [507]:
add_genre_other(top_2017)

In [508]:
del_mult_genres(top_2017)

In [509]:
top_2017

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017,pop
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017,latin
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017,latin
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017,tropical house
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017,trap
5,7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017,hip hop
6,3eR23VReFzcdmS7TYCrhC,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017,edm
7,3B54sVLJ402zGa6Xm4YGN,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017,trap
8,0KKkJNfGyhkQ5aFogxQAP,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017,pop
9,3NdDpSvN911VPGivFlV5d,I DonEEt Wanna Live Forever (Fifty Shades D...,ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017,pop


In [471]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)
# search by artist name
result = sp.track('0TK2YIli7K1leLovkQiNik')
result['popularity']

95

In [283]:
top_2019_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019,pop
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019,latin
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019,pop
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019,pop
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019,rap
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019,pop
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019,other
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019,pop
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019,pop rap
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019,pop


In [285]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

for i, track_id in top_2018['id'].iteritems():        
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by artist name
    result = sp.track(track_id)
    print(result['popularity'])

SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/tracks/6DCZcSspjsKoFjzjrWoCd:
 invalid id

In [511]:
top_2017

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017,pop
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017,latin
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017,latin
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017,tropical house
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017,trap
5,7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017,hip hop
6,3eR23VReFzcdmS7TYCrhC,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017,edm
7,3B54sVLJ402zGa6Xm4YGN,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017,trap
8,0KKkJNfGyhkQ5aFogxQAP,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017,pop
9,3NdDpSvN911VPGivFlV5d,I DonEEt Wanna Live Forever (Fifty Shades D...,ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017,pop


In [500]:
def fix_id(track, artist):
    ''' searches by track, pulls 10 albums
        if artist in album matches artist param,
        RETURNS: [artist, track, song-id]'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name
    result = sp.search(q=track, type='track')

    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = result['tracks']['items'][0]['id']
            return value
    # possibly multiple artists, if so, return first id value
    value = result['tracks']['items'][0]['id']
    return value

In [512]:
def search_by_artist2(artist, track):
    ''' returns correct track by artist to use in
        search_by_track'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']

    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        print(possible_song)
        # matches enough, match found
        if track[:2] == possible_song[:2] or track[-3:] == possible_song[-3:]:
            track = possible_song
            return track

In [513]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

def fix_dataframe_values(dataframe):
    ''' corrects IDs , incorrect tracks '''
    for i, column in dataframe.iterrows():  
        track = column['name']
        artist = column['artists']
        try:
            correct_id = fix_id(track,artist)
            print('correct id', correct_id)
            if correct_id != None:
                dataframe.loc[i, 'id'] = correct_id
            if correct_id == None:
                sp = spotipy.Spotify(auth=token)
                # search by track name
                result = sp.search(q=track, type='track')
                correct_id = result['tracks']['items'][0]['id']
                print("nones correct id is now: ", correct_id)
                dataframe.loc[i, 'id'] = correct_id


        except IndexError:
            print('COULDNT FIND', track, ' by ', artist)
            correct_track = search_by_artist2(artist, track)
            print('track is now', correct_track)
            if correct_track != None:
                dataframe.loc[i, 'name'] = correct_track
                correct_id = fix_id(correct_track, artist)
                dataframe.loc[i, 'id'] = correct_id
            else:
                track = track.split()[0]
                correct_id = fix_id(track,artist)
                print("found justing", correct_id)
            print()
            print()

    #top_2017.loc[i, 'id']

In [525]:
correct_track = "I Don’t Wanna Live Forever (Fifty Shades Darker)"
artist = "ZAYN"

checkid = fix_id(correct_track, artist)

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
# create spotipy object
sp = spotipy.Spotify(auth=token)
# search by track name
result = sp.search(q=correct_track, type='track')
result['tracks']['items'][0]['id']

'55n9yjI6qqXh5F2mYvUc2y'

In [515]:
top_2017

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,7qiZfU4dY1lWllzX7mPBI3,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017,pop
1,6rPO02ozF3bM7NnOV4h6s2,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017,latin
2,6habFhsOp2NvshLv26DqMb,Despacito,Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017,latin
3,6RUKPb4LETWmmr3iAEQktW,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017,tropical house
4,7795WJLVKJoAyVoOtCWqXN,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017,trap
5,7KXjTSCq5nL1LoYtL7XAwS,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017,hip hop
6,12GEpg2XOPyqk03JZEZnJs,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017,edm
7,3B54sVLJ402zGa6Xm4YGNe,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017,trap
8,0KKkJNfGyhkQ5aFogxQAPU,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017,pop
9,None,I Don’t Wanna Live Forever (Fifty Shades Darker),ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017,pop


In [332]:
sp = spotipy.Spotify(auth=token)
# search by track name
track = 'Ric Flair Drip (& Metro Boomin)'
result = sp.search(q=track, type='track')
artist = (result['tracks']['items'][0]['artists'][0]['name'])
correct_id = (result['tracks']['items'][0]['id'])
print(artist)
print(correct_id)

Offset
7sO5G9EABYOXQKNPNiE9NR


In [483]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

artist = 'Ozuna'
result = sp.search(q=artist, type='artist', limit = 30)
result['artists']['items']


#print(result['tracks']['items'][16]['artists'])
# artist_id = result['artists']['items'][0]['id']
# top_tracks = sp.artist_top_tracks(artist_id)
# top_10 = top_tracks['tracks']
# for song in top_10:
#     print(song['name'])

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1i8SpTcr7yvPOmcqrbnVXY'},
  'followers': {'href': None, 'total': 20893775},
  'genres': ['latin', 'puerto rican pop', 'reggaeton', 'trap latino'],
  'href': 'https://api.spotify.com/v1/artists/1i8SpTcr7yvPOmcqrbnVXY',
  'id': '1i8SpTcr7yvPOmcqrbnVXY',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/f9ede0c1b199df8dfc4a29b784342e9af2e15a2c',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ee454051d89dd99afbbe6a3beae48bf702d2cf3c',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/1f9c3f6fcf65305eb5476944efb6aecf1621745c',
    'width': 160}],
  'name': 'Ozuna',
  'popularity': 98,
  'type': 'artist',
  'uri': 'spotify:artist:1i8SpTcr7yvPOmcqrbnVXY'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/2OJ8svFeHnPIaGZCW6Zc69'},
  'followers': {'href': None, 'total': 407},
  'genres': [],
  'href': 'https://api.spotify.com/v1/artists/2OJ8svFeHnPIaGZ

In [428]:
top_2017

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017
5,7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017
6,3eR23VReFzcdmS7TYCrhC,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017
7,3B54sVLJ402zGa6Xm4YGN,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017
8,0KKkJNfGyhkQ5aFogxQAP,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017
9,3NdDpSvN911VPGivFlV5d,I DonEEt Wanna Live Forever (Fifty Shades D...,ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017
